# A-Root

Let us analyse A-Root first. From the following Figure, we see that convergence level of A-Root suddenly increases on January 2014.

![alt text](../analysis/figs/png/convergence_over_time_a.png)

by the way, what is the exact date when it happened?

In [1]:
%matplotlib inline
from ggplot import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import csv
from pandas import DataFrame
from pymongo import MongoClient, ASCENDING, DESCENDING
from bson.code import Code

In [2]:
client = MongoClient()
anycast = client.anycast_monitoring

root_list = 'acdfijklm'

reducer = Code("""
    function(obj, prev) {
        if(obj.path4.toString() != obj.path6.toString()) {
            prev.different++;
        } else {
            prev.identical++;
        }
    }
"""
)

def dateparse (time_in_secs):    
    return datetime.fromtimestamp(float(time_in_secs))

container2 = dict()

for root in root_list:
    coll = anycast['{}_root'.format(root)]
    res = coll.group(key={'timestamp':1}, condition={}, initial={'identical': 0, 'different': 0}, reduce=reducer)
    container2[root] = DataFrame(res)
    
    container2[root]['timestamp'] = pd.to_datetime(container2[root]['timestamp'] * 1000000000)
    container2[root] = container2[root].set_index(['timestamp'])
    
    container2[root]['total'] = pd.Series()
    container2[root]['diff_pct'] = pd.Series()
    container2[root]['ident_pct'] = pd.Series()

    for item in container2[root].iterrows():
        item[1]['total'] = item[1]['different'] + item[1]['identical']
        item[1]['diff_pct'] = item[1]['different'] / item[1]['total'] * 100
        item[1]['ident_pct'] = item[1]['identical'] / item[1]['total']

Identical VPs per date

In [3]:
for item in container2['a'].iterrows():
    print('{}\t\t{:.2f}'.format(item[0], item[1]['ident_pct'] * 100))

2008-05-01 00:00:00		0.00
2008-07-01 00:00:00		0.00
2009-03-01 00:00:00		0.00
2009-07-01 00:00:00		0.00
2009-08-01 00:00:00		0.00
2009-09-01 00:00:00		0.00
2009-10-01 00:00:00		0.00
2009-11-01 00:00:00		0.00
2009-12-01 00:00:00		0.00
2010-01-01 00:00:00		0.00
2010-02-01 00:00:00		0.00
2010-03-01 00:00:00		0.00
2010-04-01 00:00:00		0.00
2010-05-01 00:00:00		0.00
2010-06-01 00:00:00		0.00
2010-07-01 00:00:00		0.00
2010-08-01 00:00:00		0.00
2010-09-01 00:00:00		0.00
2010-10-01 00:00:00		0.00
2010-11-01 00:00:00		0.00
2010-12-01 00:00:00		0.00
2011-01-01 00:00:00		0.00
2011-02-01 00:00:00		0.00
2011-03-01 00:00:00		0.00
2011-04-01 00:00:00		0.00
2011-05-01 00:00:00		0.00
2011-06-01 00:00:00		0.00
2011-07-01 00:00:00		0.00
2011-08-01 00:00:00		0.00
2011-09-01 00:00:00		0.00
2011-10-01 00:00:00		0.00
2011-11-01 00:00:00		0.00
2011-12-01 00:00:00		0.00
2012-01-01 00:00:00		0.00
2012-02-01 00:00:00		0.00
2012-03-01 00:00:00		0.00
2012-04-01 00:00:00		0.00
2012-05-01 00:00:00		0.00
2012-06-01 0

Apparently, the sudden changes happened in February 2014

Now, focus on data of Jan and Feb 2014.

Q: how many mutual VPs that do not experience IPv6 path changes from jan to feb 2014?

# The following is not very clear. This is just a rough idea

In [13]:
# 1388534400 --> Jan 2014
# 1391212800 --> Feb 2014

# first, find mutual peers that exist in both date

data_jan = dict()
data_feb = dict()
peer_jan = set()
peer_feb = set()
for item in anycast['a_root'].find({'timestamp': 1388534400}):
    data_jan[item['peer']] = item
    peer_jan.add(item['peer'])
for item in anycast['a_root'].find({'timestamp': 1391212800}):
    data_feb[item['peer']] = item
    peer_feb.add(item['peer'])

mutual_peers = peer_jan & peer_feb

mutual_peers = list(mutual_peers)
print('\nMutual VPs exist in both Feb and Jan 2014: {}\n-------'.format(len(mutual_peers)))

# now find out which mutual peers experience changes from january to feb
print('VPs with changes in IPv4 path are the following:\n')
counter = 0
for peer in mutual_peers:
    if data_jan[peer]['path4'] != data_feb[peer]['path4']:
        print('{}: {} {}'.format(peer, data_jan[peer]['path4'], data_feb[peer]['path4']))
#         pass
    else:
        counter += 1
print('\nThus, mutual peers without changes in IPv4 path: {}\n---------'.format(counter))

print('VPs with changes in IPv6 path are the following:\n')
counter = 0
for peer in mutual_peers:
    if data_jan[peer]['path6'] != data_feb[peer]['path6']:
        print('{}: {} {}'.format(peer, data_jan[peer]['path6'], data_feb[peer]['path6']))
#         pass
    else:
        counter += 1
print('\nThus mutual peers without changes in IPv6 path: {}'.format(counter))


Mutual VPs exist in both Feb and Jan 2014: 60
-------
VPs with changes in IPv4 path are the following:

1280: [1280, 7342, 26415, 36620] [1280, 7342, 36620]
8596: [8596, 7342, 26415, 36625] [8596, 286, 7342, 26415, 36625]
7575: [7575, 7342, 26415, 36620] [7575, 7342, 36620]
57381: [57381, 50304, 7342, 26415, 36625] [57381, 42708, 7342, 26415, 36625]
22652: [22652, 6453, 7342, 26415, 36619] [22652, 6453, 7342, 36619]
2497: [2497, 7342, 26415, 36620] [2497, 7342, 36620]
19151: [19151, 7342, 26415, 36620] [19151, 7342, 36620]

Thus, mutual peers without changes in IPv4 path: 53
---------
VPs with changes in IPv6 path are the following:

1280: [1280, 3356, 26415, 36620] [1280, 7342, 36620]
25091: [25091, 7342, 26415, 36623] [25091, 7342, 26415, 36625]
8596: [8596, 7342, 36622, 26415] [8596, 6939, 7342, 36622, 26415]
6679: [6679, 3356, 26415, 36620] [6679, 6939, 7342, 36622, 26415]
7575: [7575, 2914, 36625] [7575, 6939, 7342, 36620]
286: [286, 2914, 36625] [286, 2914, 36619]
20640: [20640,

--------------------------------------------------
This following is the correct one

In [11]:
print('total peers in jan: {}'.format(len(peer_jan)))
print('total peers in feb: {}'.format(len(peer_feb)))
print()

# identical peer di jan
counter = 0
for peer in data_jan:
    if data_jan[peer]['path4'] == data_jan[peer]['path6']:
        counter += 1
print('identical peers in jan: {}'.format(counter))

# identical peer di feb
counter = 0
for peer in data_feb:
    if data_feb[peer]['path4'] == data_feb[peer]['path6']:
        counter += 1
print('number of identical VPs in feb: {}'.format(counter))
print()
      
# peer yang baru muncul di bulan feb
additional_peers = []

for peer in peer_feb:
    if peer not in peer_jan:
        additional_peers.append(peer)

print('number of additional VPs in February: {}'.format(additional_peers))

# peer yang ilang di bulan feb
removed_peers = []

for peer in peer_jan:
    if peer not in peer_feb:
        removed_peers.append(peer)

print('number of removed VPs in February: {}'.format(removed_peers))
print()

# Peers yang pada bulan Jan punya path yg berbeda, kemudian di bulan Feb path nya jadi sama
converged_peers = list()
counter = 0
for peer in mutual_peers:
    if (data_jan[peer]['path4'] != data_jan[peer]['path6']) and (data_feb[peer]['path4'] == data_feb[peer]['path6']):
#         print(peer)
        counter += 1
        converged_peers.append(peer)
print('number of VPs that in Jan have different path, but in feb have identical path: {}'.format(counter))

# focus on converged peers
print()
print('converged peers:')
for count, peer in enumerate(converged_peers):
    print('<{}> {} - {} {} {} {}'.format(count + 1, peer,data_jan[peer]['path4'],data_jan[peer]['path6'],data_feb[peer]['path4'],data_feb[peer]['path6']))

# converging peers due to changing Root ASN destination. Initially, on Jan, they have similar path except for the A-Root ASN.
# then, in feb
counter = 0
print()
print('converging VPs due to changes in Root ASN destination:')
# for peer in mutual_peers:
for peer in converged_peers:    
    if data_jan[peer]['path4'] != data_jan[peer]['path6'] and data_jan[peer]['path4'][:-1] == data_jan[peer]['path6'][:-1]:
        counter += 1
        print('<{}> {}: \n\t January: {} {} \n\tFebruary: {} {}'.format(counter, peer, data_jan[peer]['path4'], data_jan[peer]['path6'], data_feb[peer]['path4'], data_feb[peer]['path6']))

total peers in jan: 60
total peers in feb: 64

identical peers in jan: 1
number of identical VPs in feb: 23

number of additional VPs in February: [22548, 52888, 16735, 1916]
number of removed VPs in February: []

number of VPs that in Jan have different path, but in feb have identical path: 18

converged peers:
<1> 1280 - [1280, 7342, 26415, 36620] [1280, 3356, 26415, 36620] [1280, 7342, 36620] [1280, 7342, 36620]
<2> 25091 - [25091, 7342, 26415, 36625] [25091, 7342, 26415, 36623] [25091, 7342, 26415, 36625] [25091, 7342, 26415, 36625]
<3> 31019 - [31019, 7342, 26415, 36625] [31019, 7342, 26415, 36623] [31019, 7342, 26415, 36625] [31019, 7342, 26415, 36625]
<4> 12859 - [12859, 7342, 26415, 36625] [12859, 7342, 26415, 36623] [12859, 7342, 26415, 36625] [12859, 7342, 26415, 36625]
<5> 12989 - [12989, 7342, 26415, 36625] [12989, 7342, 26415, 36623] [12989, 7342, 26415, 36625] [12989, 7342, 26415, 36625]
<6> 2497 - [2497, 7342, 26415, 36620] [2497, 7342, 36624, 26415] [2497, 7342, 36620] 

Quick verdict:
1. mutual peers do not experience changes in their IPv4 path
2. (IPv4) Starting from Jan 2014, A-Root originating ASNs started to directly peer with ASN 7342. Previously, they peered to ASN 26415
3. There are situations where the IPv4/IPv6 AS path are similar, *until* it reaches the originating ASN (IPv4 to 36625, IPv6 to 36623). This practice is employed by ASN 26415 as the upstream provider for A-Root in January. Since Feb 2014, most of them both IPv4 and IPv6 routes are directed towards 36625. This is the majority case for converging peers
4. A-Root started using anycast since February 2008, together with 5 other Root Servers
5. A-Root prefixes are announced from multiple ASNs. The motivation and how they differentiate it is not known. Possibly, each instance uses its own dedicated ASN. However, there are 7 ASNs as opposed to 5 instances. One ASN (64820) is listed as private use only (what does it mean?)
6. (see the few next note) One factor contributing to low convergence level of A-Root is different announcing ASNs for IPv4 and IPv6 experienced by VPs. And it still happens recently, although not that much anymore

## Conclusion
The immediate increase of converging level is largely because there is changes on their prefix announcement policy. Previously, there was difference between IPv4 route and IPv6 rote where some IPv4 routes were directed towards ASN 36625 and IPv6 towards 36623. Starting on February 2014, ASN 36623 was no more announcing IPv6 prefix.


In [55]:
print('is it true that there is no more IPv6 route towards ASN 36623?')

for peer in data_feb:
    if data_feb[peer]['path6'][-1] == 36623:
        print('{} - {}'.format(peer, data_feb[peer]['path6']))

print('finish')

is it true that there is no more IPv6 route towards ASN 36623?
finish


In [71]:
for peer in converged_peers:
    if data_jan[peer]['path4'][-1] == 36625 and data_jan[peer]['path6'][-1] == 36623:
        print('{}: {}\t{} {}'.format(peer, data_jan[peer]['collector'], data_jan[peer]['path4'], data_jan[peer]['path6']))

25091: 13	[25091, 7342, 26415, 36625] [25091, 7342, 26415, 36623]
31019: 03	[31019, 7342, 26415, 36625] [31019, 7342, 26415, 36623]
12859: 03	[12859, 7342, 26415, 36625] [12859, 7342, 26415, 36623]
12989: 03	[12989, 7342, 26415, 36625] [12989, 7342, 26415, 36623]
20932: 04	[20932, 7342, 26415, 36625] [20932, 7342, 26415, 36623]
15435: 03	[15435, 7342, 26415, 36625] [15435, 7342, 26415, 36623]
50763: 03	[50763, 7342, 26415, 36625] [50763, 7342, 26415, 36623]
1103: 03	[1103, 7342, 26415, 36625] [1103, 7342, 26415, 36623]
12637: 10	[12637, 7342, 26415, 36625] [12637, 7342, 26415, 36623]
28917: 13	[28917, 7342, 26415, 36625] [28917, 7342, 26415, 36623]
50300: 00	[50300, 7342, 26415, 36625] [50300, 7342, 26415, 36623]


In [81]:
# which ASN originated IPv6 prefix?

for peer in data_feb:
    print('{}: \toriginating ASN: {}\tCollector: {}'.format(peer, data_feb[peer]['path6'][-1], data_feb[peer]['collector']))

1280: 	originating ASN: 36620	Collector: 14
6720: 	originating ASN: 26415	Collector: 05
25091: 	originating ASN: 36625	Collector: 13
25220: 	originating ASN: 26415	Collector: 12
6667: 	originating ASN: 26415	Collector: 07
4589: 	originating ASN: 36620	Collector: 12
5396: 	originating ASN: 26415	Collector: 10
6679: 	originating ASN: 26415	Collector: 13
52888: 	originating ASN: 36619	Collector: 15
29636: 	originating ASN: 36625	Collector: 01
56730: 	originating ASN: 36625	Collector: 01
286: 	originating ASN: 36619	Collector: 01
8607: 	originating ASN: 36625	Collector: 01
20640: 	originating ASN: 26415	Collector: 12
37989: 	originating ASN: 36619	Collector: 00
7575: 	originating ASN: 36620	Collector: 14
57381: 	originating ASN: 36625	Collector: 00
48166: 	originating ASN: 26415	Collector: 12
51185: 	originating ASN: 26415	Collector: 10
680: 	originating ASN: 26415	Collector: 12
553: 	originating ASN: 26415	Collector: 12
9002: 	originating ASN: 26415	Collector: 13
29611: 	originating ASN: 

In [90]:
df = DataFrame(data_feb)
df = df.T
df = df.drop(['timestamp', '_id'], 1)
grouped_df = df.groupby(['collector'])
for item in grouped_df:
    print(item)

('00',       collector                               path4  \
6881         00   [6881, 29208, 7342, 26415, 36625]   
8758         00                 [8758, 3356, 36622]   
15469        00                [15469, 3356, 36622]   
22652        00          [22652, 6453, 7342, 36619]   
29049        00               [29049, 20485, 36631]   
29608        00                [29608, 3356, 36622]   
37989        00          [37989, 4844, 2914, 36619]   
50300        00         [50300, 7342, 26415, 36625]   
57381        00  [57381, 42708, 7342, 26415, 36625]   
57821        00  [57821, 48039, 7342, 26415, 36625]   

                                    path6   peer  
6881     [6881, 6939, 7342, 36622, 26415]   6881  
8758     [8758, 6939, 7342, 36622, 26415]   8758  
15469   [15469, 6939, 7342, 36622, 26415]  15469  
22652          [22652, 6939, 7342, 36620]  22652  
29049         [29049, 7342, 36622, 26415]  29049  
29608   [29608, 6939, 7342, 26415, 36625]  29608  
37989          [37989, 4844, 2

## Which peers that have different destination ASN for IPv4 and IPv6 routes?

In [92]:
for peer in data_jan:
    if data_jan[peer]['path4'][-1] != data_jan[peer]['path6'][-1]:
        print('{}:\t{} {} \t\t{} {}'.format(peer, 
                                            data_jan[peer]['path4'][-1], 
                                            data_jan[peer]['path6'][-1],
                                            data_feb[peer]['path4'][-1],
                                            data_feb[peer]['path6'][-1]))

6720:	36625 36620 		36625 26415
25091:	36625 36623 		36625 36625
25220:	36625 26415 		36625 26415
6667:	36625 26415 		36625 26415
5396:	36622 26415 		36622 26415
6679:	36625 36620 		36625 26415
8607:	36622 36625 		36622 36625
20640:	36625 36620 		36625 26415
7575:	36620 36625 		36620 36620
57381:	36625 36623 		36625 36625
48166:	36625 26415 		36625 26415
51185:	36625 26415 		36625 26415
680:	36622 36620 		36622 26415
553:	36625 26415 		36625 26415
9002:	36625 36623 		36625 26415
57821:	36625 26415 		36625 26415
13237:	36625 36623 		36625 26415
8758:	36622 26415 		36622 26415
12859:	36625 36623 		36625 36625
12989:	36625 36623 		36625 36625
28917:	36625 36623 		36625 36625
25152:	36619 36625 		36619 36619
2497:	36620 26415 		36620 36620
50763:	36625 36623 		36625 36625
20932:	36625 36623 		36625 36625
49605:	36625 26415 		36625 26415
15435:	36625 36623 		36625 36625
19151:	36620 26415 		36620 36620
29140:	36625 26415 		36625 26415
29049:	36631 36620 		36631 26415
31019:	36625 36623 		36

## Does this practice (different announcing ASN for IPv4/IPv6 prefixes) still happen recently?

In [93]:
for item in anycast['a_root'].find({'timestamp': 1464739200}):
    if item['path4'][-1] != item['path6'][-1]:
        print('{}:\t{} {}'.format(item['peer'], item['path4'][-1], item['path6'][-1]))

513:	36625 36622
1836:	36625 36622
3257:	36625 36631
3491:	36625 36631
4589:	36622 36619
5396:	36625 36622
6453:	36625 36619
6720:	36625 36622
9002:	36625 36622
9304:	36631 36619
13030:	36619 36622
15547:	36625 36622
15605:	36625 36622
20612:	36625 36622
29608:	36622 36625
30844:	36622 36631
34146:	36625 36622
37271:	36622 36625
41497:	36625 36622
49835:	36625 36622
50620:	36622 36625
53070:	36625 36619
196621:	36625 36619
262317:	36625 36619


In [79]:
for peer in data_jan:
    if data_feb[peer]['path6'][-1] == 36623:
#         print('true')
        print('{}:\t{} {} {} {}'.format(peer, data_jan[peer]['path4'], data_jan[peer]['path6'], data_feb[peer]['path4'], data_feb[peer]['path6']))

# D-Root

When was it happened?
![alt text](../analysis/figs/png/convergence_over_time_d.png)

In [56]:
for item in container2['d'].iterrows():
    print('{}\t\t{:.2f}'.format(item[0], item[1]['ident_pct'] * 100))

2011-07-01 00:00:00		9.76
2011-08-01 00:00:00		5.26
2011-09-01 00:00:00		10.26
2011-10-01 00:00:00		6.82
2011-11-01 00:00:00		6.98
2011-12-01 00:00:00		9.52
2012-01-01 00:00:00		13.33
2012-02-01 00:00:00		6.98
2012-03-01 00:00:00		9.09
2012-04-01 00:00:00		8.89
2012-05-01 00:00:00		6.67
2012-06-01 00:00:00		6.82
2012-07-01 00:00:00		11.90
2012-08-01 00:00:00		6.00
2012-09-01 00:00:00		9.80
2012-10-01 00:00:00		6.00
2012-11-01 00:00:00		6.90
2012-12-01 00:00:00		4.35
2013-01-01 00:00:00		6.56
2013-02-01 00:00:00		4.62
2013-03-01 00:00:00		7.69
2013-04-01 00:00:00		11.36
2013-05-01 00:00:00		4.92
2013-06-01 00:00:00		5.88
2013-07-01 00:00:00		6.15
2013-08-01 00:00:00		5.97
2013-09-01 00:00:00		5.97
2013-10-01 00:00:00		6.56
2013-11-01 00:00:00		48.48
2013-12-01 00:00:00		50.77
2014-01-01 00:00:00		49.18
2014-02-01 00:00:00		59.38
2014-03-01 00:00:00		59.38
2014-04-01 00:00:00		54.84
2014-05-01 00:00:00		53.23
2014-06-01 00:00:00		48.33
2014-07-01 00:00:00		50.85
2014-08-01 00:00:00		51.6

**Conclusion:** The sudden increase started on November 2013.

-------
Now, let's find out VPs that present in both October and November 2013. From those VPs, find out how many of them experienced changes in their IPv4 and IPv6 paths

In [21]:
# 1380585600 --> Oct 2013
# 1383264000 --> Nov 2013

# first, find mutual peers that exist in both date

data_oct = dict()
data_nov = dict()
peer_oct = set()
peer_nov = set()
for item in anycast['d_root'].find({'timestamp': 1380585600}):
    data_oct[item['peer']] = item
    peer_oct.add(item['peer'])
for item in anycast['d_root'].find({'timestamp': 1383264000}):
    data_nov[item['peer']] = item
    peer_nov.add(item['peer'])

mutual_peers = peer_oct & peer_nov

mutual_peers = list(mutual_peers)
print('total mutual VPs that present both in Oct and Nov: {}\n'.format(len(mutual_peers)))

# now find out which mutual peers experience changes from oct to nov
counter = 0
print('The following are VPs that change its IPv4 path in November:\n')
for peer in mutual_peers:
    if data_oct[peer]['path4'] != data_nov[peer]['path4']:
        print('{}: {} {}'.format(peer, data_oct[peer]['path4'], data_nov[peer]['path4']))
    else:
        counter += 1
print('\n thus, mutual peers present in both month with changes in IPv4 path: {}, without changes in IPv4 path: {}\n\n'.format(len(mutual_peers) - counter, counter))

counter = 0
print('The following are VPs that change its IPv6 path in November:\n')
for peer in mutual_peers:
    if data_oct[peer]['path6'] != data_nov[peer]['path6']:
        print('{}: {} {}'.format(peer, data_oct[peer]['path6'], data_nov[peer]['path6']))
    else:
        counter += 1
print('\n thus, mutual peers present in both month with changes in IPv6 path: {}, without changes in IPv6 path: {}\n\n'.format(len(mutual_peers) - counter, counter))



total mutual VPs that present both in Oct and Nov: 61

The following are VPs that change its IPv4 path in November:

1280: [1280, 7922, 33657, 27] [1280, 42, 27]
25220: [25220, 6939, 22925, 27] [25220, 42, 27]
6667: [6667, 6939, 22925, 27] [6667, 42, 27]
8596: [8596, 6939, 22925, 27] [8596, 42, 27]
6679: [6679, 6939, 22925, 27] [6679, 42, 27]
56730: [56730, 6939, 22925, 27] [56730, 42, 27]
286: [286, 6939, 22925, 27] [286, 2914, 42, 27]
8607: [8607, 6939, 22925, 27] [8607, 42, 27]
20640: [20640, 6939, 22925, 27] [20640, 6695, 42, 27]
57381: [57381, 42708, 6939, 22925, 27] [57381, 42708, 42, 27]
48166: [48166, 6939, 22925, 27] [48166, 42, 27]
553: [553, 680, 20965, 11537, 10886, 27] [553, 42, 27]
9002: [9002, 6939, 22925, 27] [9002, 42, 27]
29611: [29611, 6939, 22925, 27] [29611, 42, 27]
31019: [31019, 6939, 22925, 27] [31019, 42, 27]
13237: [13237, 6939, 22925, 27] [13237, 42, 27]
3257: [3257, 7922, 33657, 27] [3257, 42, 27]
12989: [12989, 6939, 22925, 27] [12989, 42, 27]
2497: [2497, 

------
Now find the converging VPs in November

In [25]:
print('total VPs in oct: {}'.format(len(peer_oct)))
print('total VPs in nov: {}'.format(len(peer_nov)))
print()

# identical peer di oct
counter = 0
for peer in data_oct:
    if data_oct[peer]['path4'] == data_oct[peer]['path6']:
        counter += 1
print('mutual VPs in oct: {}'.format(counter))

# identical peer di nov
counter = 0
for peer in data_nov:
    if data_nov[peer]['path4'] == data_nov[peer]['path6']:
        counter += 1
print('mutual VPs in nov: {}'.format(counter))
print()
      
# peer yang baru muncul di bulan nov
additional_peers = []

for peer in peer_nov:
    if peer not in peer_oct:
        additional_peers.append(peer)

print('additional VPs in November: {}'.format(additional_peers))

# peer yang ilang di bulan nov
removed_peers = []

for peer in peer_oct:
    if peer not in peer_nov:
        removed_peers.append(peer)

print('removed VPs in November: {}'.format(removed_peers))
print()

# Peers yang pada bulan Oct punya path yg berbeda, kemudian di bulan Nov path nya jadi sama
converged_peers = list()
counter = 0
for peer in mutual_peers:
    if (data_oct[peer]['path4'] != data_oct[peer]['path6']) and (data_nov[peer]['path4'] == data_nov[peer]['path6']):
#         print(peer)
        counter += 1
        converged_peers.append(peer)
print('number of VPs that in Oct have different IPv4/IPv6 paths, but in Nov have identical path: {}'.format(counter))

# focus on converged peers
shorter_count = 0
print()
print('converged peers:')
for count, peer in enumerate(converged_peers):
    print('<{}> {} \n\t October: {} {} \n\t November: {} {}'.format(count + 1, peer,data_oct[peer]['path4'],data_oct[peer]['path6'],data_nov[peer]['path4'],data_nov[peer]['path6']))
    if data_oct[peer]['path4'] > data_nov[peer]['path4'] and data_oct[peer]['path6'] > data_nov[peer]['path6']:
        shorter_count += 1
print('\nconverging VPs that experienced shorter path: {}'.format(shorter_count))


total VPs in oct: 61
total VPs in nov: 66

mutual VPs in oct: 4
mutual VPs in nov: 32

additional VPs in November: [22548, 52888, 20932, 16735, 1916]
removed VPs in November: []

number of VPs that in Oct have different IPv4/IPv6 paths, but in Nov have identical path: 26

converged peers:
<1> 1280 
	 October: [1280, 7922, 33657, 27] [1280, 11164, 10886, 27] 
	 November: [1280, 42, 27] [1280, 42, 27]
<2> 25220 
	 October: [25220, 6939, 22925, 27] [25220, 6939, 10886, 27] 
	 November: [25220, 42, 27] [25220, 42, 27]
<3> 8596 
	 October: [8596, 6939, 22925, 27] [8596, 6939, 10886, 27] 
	 November: [8596, 42, 27] [8596, 42, 27]
<4> 56730 
	 October: [56730, 6939, 22925, 27] [56730, 6939, 10886, 27] 
	 November: [56730, 42, 27] [56730, 42, 27]
<5> 8607 
	 October: [8607, 6939, 22925, 27] [8607, 6939, 10886, 27] 
	 November: [8607, 42, 27] [8607, 42, 27]
<6> 57381 
	 October: [57381, 42708, 6939, 22925, 27] [57381, 42708, 6939, 10886, 27] 
	 November: [57381, 42708, 42, 27] [57381, 42708, 42

D-Root is quite interesting, since it has 100 instances, with 18 of them are configured as global nodes. Only one instance is not IPv6-capable. However, the convergence level does not quite reflect the quantity of instances they have.

### Quick verdict from converged peers:
1. In general, the main provider to reach global Internet is ASN 22925
2. Initially, for IPv4, D-Root is identified to have two upstream provider (33657 and 22925). Then, D-Root added another upstream provider, ASN 42, which immediately became the main center of the tree.
3. Initially, for IPv6, the only upstream provider was ASN 10886. After D-Root added ASN 42 as its upstream, it also became the center of the tree.
4. To sum it up, the decision to use ASN 42 as the upstream provider for IPv4 and IPv6 significantly increases the convergence level

### Conclusions
Initially, D-Root used different providers for IPv4 and IPv6. Later, they added ASN 42 as their new upstream, and it greatly helped on converging the catchment